<a href="https://colab.research.google.com/github/kuds/rl-car-racing/blob/main/%5BCar%20Racing%5D%20Deep%20Q-Network%20(DQN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Network (DQN)

In [ ]:
!pip install swig moviepy

In [ ]:
!pip install gymnasium gymnasium[box2d] stable_baselines3

In [ ]:
import gymnasium
from gymnasium.wrappers import RecordVideo
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecFrameStack, VecVideoRecorder
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import VecTransposeImage

import numpy
import platform
import scipy
import stable_baselines3
import IPython
import matplotlib
import matplotlib.pyplot
import torch
import time

In [ ]:
# Create Training CarRacing environment
env = make_vec_env("CarRacing-v2", n_envs=1)
env = VecFrameStack(env, n_stack=4)

# Create Elevation CarRacing Environment
env_val = make_vec_env("CarRacing-v2", n_envs=1)
env_val = VecFrameStack(env_val, n_stack=4)
env_val = VecTransposeImage(env_val)

eval_callback = EvalCallback(env_val, best_model_save_path="./logs/", log_path="./logs/", eval_freq=5000, render=False)

# Initialize DQN
model = DQN('CnnPolicy', env, verbose=0)

# Train the model
model.learn(total_timesteps=10000, progress_bar=True)

# Save the model
model.save("dqn_car_racing")

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=15)
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
# Create Elevation CarRacing environment
env = make_vec_env("CarRacing-v2", n_envs=1, seed=0)

# Load the best model
best_model_path = "./logs/best_model.zip"
best_model = SAC.load(best_model_path, env=env)

mean_reward, std_reward = evaluate_policy(best_model, env, n_eval_episodes=15)
print(f"Best Model - Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

# Record video of the best model playing CarRacing
env = VecVideoRecorder(env, "./videos/", video_length=10000, record_video_trigger=lambda x: x == 0, name_prefix="best_model_car_racing_sac")

obs = env.reset()
for _ in range(10000):
    action, _states = best_model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()
    if dones:
        break

env.close()